In [ ]:
import sys

import numpy as np
sys.path.append("../")

In [ ]:
from functools import reduce
from abc import ABC, abstractmethod

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import numpy as np

class DenseTied(tfkl.Layer):
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 tied_to=None,
                 **kwargs):
        self.tied_to = tied_to
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super().__init__(**kwargs)
        self.units = units
        self.activation = tfk.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = tfk.initializers.get(kernel_initializer)
        self.bias_initializer = tfk.initializers.get(bias_initializer)
        self.kernel_regularizer = tfk.regularizers.get(kernel_regularizer)
        self.bias_regularizer = tfk.regularizers.get(bias_regularizer)
        self.activity_regularizer = tfk.regularizers.get(activity_regularizer)
        self.kernel_constraint = tfk.constraints.get(kernel_constraint)
        self.bias_constraint = tfk.constraints.get(bias_constraint)
        self.input_spec = tfkl.InputSpec(min_ndim=2)
        self.supports_masking = True
                
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        if self.tied_to is not None:
            self.kernel = tf.transpose(self.tied_to.kernel)
            self._non_trainable_weights.append(self.kernel)
        else:
            self.kernel = self.add_weight(shape=(input_dim, self.units),
                                          initializer=self.kernel_initializer,
                                          name='kernel',
                                          regularizer=self.kernel_regularizer,
                                          constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        #self.input_spec = tfkl.InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel)
        if self.use_bias:
            output = tf.nn.bias_add(output, self.bias, data_format='N..C')
        if self.activation is not None:
            output = self.activation(output)
        return output

class DenseEncoder(tfkl.Layer):
    """Fully connected encoder made of 'Dense' layers."""
    def __init__(self,
                 encoding_dim,
                 units_list,
                 hidden_activation,
                 output_activation,
                 weight_reg=None,
                 bias_reg=None,
                 output_reg=None,
                 dtype=None,
                 name=None):
        """Crates a DenseEncoder object.

        Args:
          encoding_dim: The number of nodes in the encoder output.
          unitsl_list: List of integers. For every list element a Dense layer
             will be crated and the number of nodes will be set to the value 
             of the element
          hidden_activation: The activation funtion used for all the layers 
            apart from the output layer.
          output_activation: The activation functiun used for the output layer
          weight_reg: 'kernel_regularizer' of all layers.
          bias_reg: 'bias_regularizer' of all layers.
          output_reg: 'activity_regularizer' for all layers.
          dtype: Data type, this argument is passed to the parent 'Layer' 
            class constructor
          name: Object name, string to be passed to the parent class 
            constructor
        """
        super(DenseEncoder, self).__init__(name=name, dtype=dtype)
        # Create the list of the hidden layers
        self.hidden_layers = [
            tfkl.Dense(units, activation=hidden_activation, dtype=dtype,
                       kernel_regularizer=weight_reg,
                       bias_regularizer=bias_reg,
                       activity_regularizer=output_reg) 
            for units 
            in units_list]
        
        # Creates the output layer
        self.output_layer = tfkl.Dense(encoding_dim, dtype=dtype,
                                       activation=output_activation,
                                       kernel_regularizer=weight_reg,
                                       bias_regularizer=bias_reg,
                                       activity_regularizer=output_reg)

    def call(self, inputs):
        """Runs the inputs through all of encoder layers"""
        activation = reduce(lambda x, layer: layer(x), self.hidden_layers, 
                            inputs)
        return self.output_layer(activation)


class DenseDecoder(tfkl.Layer):
    """Fully connected decoders made of 'Dense' layers."""
    def __init__(self,
                 output_dim,
                 units_list,
                 hidden_activation,
                 output_activation,
                 encoder_tied=None,
                 weight_reg=None,
                 bias_reg=None,
                 output_reg=None,
                 dtype=None,
                 name=None):
        """Crates a DenseDecoder object.

        Args:
          encoding_dim: The number of nodes in the decoder output.
          unitsl_list: List of integers. For every list element a Dense layer
             will be crated and the number of nodes will be set to the value 
             of the element
          hidden_activation: The activation funtion used for all the layers 
            apart from the output layer.
          output_activation: The activation functiun used for the output layer
          weight_reg: 'kernel_regularizer' of all layers.
          bias_reg: 'bias_regularizer' of all layers.
          output_reg: 'activity_regularizer' for all layers.
          dtype: Data type, this argument is passed to the parent 'Layer' 
            class constructor
          name: Object name, string to be passed to the parent class 
            constructor
        """
        super(DenseDecoder, self).__init__(name=name, dtype=dtype)
        if not encoder_tied:
          # Creates the list of hidden layers 
          self.hidden_layers = [
              tfkl.Dense(units, activation=hidden_activation, dtype=dtype,
                        kernel_regularizer=weight_reg,
                        bias_regularizer=bias_reg,
                        activity_regularizer=output_reg) 
              for units 
              in units_list] 

          # Creates the output layer
          self.output_layer = tfkl.Dense(output_dim, dtype=dtype,
                                        activation=output_activation,
                                        kernel_regularizer=weight_reg,
                                        bias_regularizer=bias_reg,
                                        activity_regularizer=output_reg)
        else:
          # Creates the list of hidden layers 
          n_layers = len(units_list)
          self.hidden_layers = [
              DenseTied(units, activation=hidden_activation, dtype=dtype,
                        kernel_regularizer=weight_reg,
                        bias_regularizer=bias_reg,
                        activity_regularizer=output_reg,
                        tied_to=encoder_tied.hidden_layers[n_layers-i-1]
                                if i != n_layers-1 else encoder_tied.output_layer) 
              for i, units 
              in enumerate(units_list)] 

          # Creates the output layer
          self.output_layer = DenseTied(output_dim, dtype=dtype,
                                        activation=output_activation,
                                        kernel_regularizer=weight_reg,
                                        bias_regularizer=bias_reg,
                                        activity_regularizer=output_reg,
                                        tied_to=encoder_tied.hidden_layers[0])
    def call(self, inputs):
        """Run the inputs through all of the decoder layers."""
        activation = reduce(lambda x, layer: layer(x), self.hidden_layers, 
                            inputs)
        return self.output_layer(activation)

In [ ]:
from pae.models.autoencoder import DenseAutoencoder, DenseAutoencoderTied
from pae.models.flows import MAF
from pae.models.nn import PaeBuilder

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras as tfk

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
builder = PaeBuilder()

ae_config = {
    'input_dim':10, 
    'encoding_dim':3, 
    'units':[8, 5],
    'weight_reg':tfk.regularizers.L1L2(l1=1e-5, l2=1e-4),
    'output_activation':tf.nn.sigmoid
}
nf_config = {
    'n_dims':3, 
    'n_layers':2, 
    'units':[32 for _ in range(4)]
}
optimizer_ae = {
    'learning_rate': 0.001
}
optimizer_nf = {
    'learning_rate': 0.005
}

builder.make_ae_model(DenseAutoencoderTied, ae_config)
builder.make_ae_optimizer(tfk.optimizers.Adam, optimizer_ae)
builder.make_nf_model(MAF, nf_config)
builder.make_nf_optimizer(tfk.optimizers.Adam, optimizer_nf)
builder.compile_ae()
builder.compile_nf()
pae = builder.pae

In [ ]:
ae_train ={
    'batch_size':200,
    'epochs':120,
    "verbose":0
}

nf_train ={
    'batch_size':200,
    'epochs':80,
    "verbose":0
}



In [ ]:
dataset = np.random.normal(0, 1, (1000,10))


In [ ]:
with tf.device("/device:GPU:0"):
    %time pae.fit(dataset,None,ae_train,nf_train)

In [ ]:
for l in pae.ae.encoder.hidden_layers:
    print(l.units)